# Хакатон: Прогноз доходов (DS Pipeline)

**Улучшения в этой версии:**
1. Логарифмирование целевой переменной (`np.log1p`).
2. Удаление признаков с нулевой важностью.
3. Тюнинг гиперпараметров CatBoost.
4. Экспорт артефактов для Docker-сервиса.

In [1]:
# 1. Установка библиотек
!pip install catboost shap pandas scikit-learn numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
import json
import os

# Метрика WMAE
def weighted_mean_absolute_error(y_true, y_pred, weights):
    return (weights * np.abs(y_true - y_pred)).mean()

In [3]:
# 2. Загрузка данных
print("Загрузка данных...")
try:
    train_df = pd.read_csv('hackathon_income_train.csv', decimal=',', sep=';', low_memory=False)
    test_df = pd.read_csv('hackathon_income_test.csv', decimal=',', sep=';', low_memory=False)
    print(f"Train shape: {train_df.shape}")
    print(f"Test shape: {test_df.shape}")
except FileNotFoundError:
    print("ОШИБКА: Не найдены файлы csv. Загрузите их в сессию Colab!")

Загрузка данных...
Train shape: (76786, 224)
Test shape: (73214, 222)


In [4]:
# 3. Список признаков для удаления (Noise Features)
# Эти признаки показали 0 важности на предыдущих тестах
USELESS_FEATURES = [
    'addrref', 'city_smart_name', 'dp_ewb_last_employment_position',
    'client_active_flag', 'vert_has_app_ru_tinkoff_investing',
    'dp_ewb_dismissal_due_contract_violation_by_lb_cnt', 'period_last_act_ad',
    'ovrd_sum', 'businessTelSubs', 'dp_ils_days_ip_share_5y',
    'nonresident_flag', 'vert_has_app_ru_vtb_invest',
    'hdb_bki_total_pil_cnt', 'accountsalary_out_flag',
    'id', 'dt' # Служебные тоже удаляем
]

In [5]:
# 4. Функция предобработки
def preprocess_data(df, is_train=True):
    df_proc = df.copy()

    # Удаляем мусорные колонки
    df_proc = df_proc.drop(columns=USELESS_FEATURES, errors='ignore')

    # --- Обработка смешанных типов ---
    # Пытаемся превратить object колонки в числа, если это возможно
    object_cols = df_proc.select_dtypes(include='object').columns
    for col in object_cols:
        # Если уникальных значений много, скорее всего это число (зарплата, оборот) записанное как строка
        if df_proc[col].nunique() > 100:
            try:
                # Убираем пробелы (разделители тысяч) и меняем запятую на точку
                temp_col = df_proc[col].astype(str).str.replace(' ', '').str.replace(',', '.')
                df_proc[col] = pd.to_numeric(temp_col, errors='coerce')
            except:
                pass

    # --- Заполнение пропусков ---
    # Числовые -> 0
    num_cols = df_proc.select_dtypes(include=['float64', 'int64']).columns
    df_proc[num_cols] = df_proc[num_cols].fillna(0)

    # Категориальные -> "MISSING"
    cat_cols = df_proc.select_dtypes(include=['object']).columns
    df_proc[cat_cols] = df_proc[cat_cols].fillna("MISSING")

    if is_train:
        y = df_proc['target']
        w = df_proc['w']
        # Удаляем таргет и веса из фичей
        X = df_proc.drop(columns=['target', 'w'], errors='ignore')
        return X, y, w
    else:
        X = df_proc.drop(columns=['target', 'w'], errors='ignore')
        return X

print("Препроцессинг данных...")
X, y, w = preprocess_data(train_df, is_train=True)
X_submit = preprocess_data(test_df, is_train=False)

# Сохраняем списки для использования в сервисе
feature_names = list(X.columns)
cat_features = list(X.select_dtypes(include=['object']).columns)

print(f"Осталось признаков: {len(feature_names)}")
print(f"Категориальных: {len(cat_features)}")

Препроцессинг данных...
Осталось признаков: 206
Категориальных: 10


In [6]:
# 5. Подготовка к обучению (Log Target)

# Делим на обучение и валидацию
X_train, X_val, y_train, y_val, w_train, w_val = train_test_split(
    X, y, w, test_size=0.2, random_state=42
)

# !!! ВАЖНО: Логарифмируем таргет !!!
# Это сжимает хвосты распределения и уменьшает ошибку
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

# Создаем пулы данных CatBoost
train_pool = Pool(X_train, y_train_log, cat_features=cat_features, weight=w_train)
val_pool = Pool(X_val, y_val_log, cat_features=cat_features, weight=w_val)

In [7]:
# 6. Обучение модели

model = CatBoostRegressor(
    iterations=2500,          # Больше итераций
    learning_rate=0.02,       # Меньше шаг -> выше точность
    depth=7,                  # Глубина дерева
    l2_leaf_reg=5,            # Регуляризация
    loss_function='RMSE',     # Оптимизируем RMSE на логарифмах (стабильнее)
    eval_metric='MAE',        # Следим за MAE
    random_seed=42,
    verbose=250,
    early_stopping_rounds=200,
    allow_writing_files=False,
    task_type="CPU"           # Поставь GPU, если включил в настройках Colab
)

print("Start Training...")
model.fit(train_pool, eval_set=val_pool)

Start Training...
0:	learn: 0.9502501	test: 0.9612849	best: 0.9612849 (0)	total: 326ms	remaining: 13m 33s
250:	learn: 0.4799405	test: 0.4965639	best: 0.4965639 (250)	total: 1m 14s	remaining: 11m 10s
500:	learn: 0.4477460	test: 0.4706419	best: 0.4706419 (500)	total: 2m 27s	remaining: 9m 47s
750:	learn: 0.4260704	test: 0.4559532	best: 0.4559532 (750)	total: 3m 39s	remaining: 8m 30s
1000:	learn: 0.4099132	test: 0.4477719	best: 0.4477644 (999)	total: 4m 50s	remaining: 7m 15s
1250:	learn: 0.3982156	test: 0.4433079	best: 0.4433079 (1250)	total: 6m 3s	remaining: 6m 3s
1500:	learn: 0.3882492	test: 0.4398567	best: 0.4398534 (1498)	total: 7m 15s	remaining: 4m 49s
1750:	learn: 0.3793782	test: 0.4375168	best: 0.4375168 (1750)	total: 8m 32s	remaining: 3m 39s
2000:	learn: 0.3714299	test: 0.4355338	best: 0.4355338 (2000)	total: 9m 46s	remaining: 2m 26s
2250:	learn: 0.3640836	test: 0.4338100	best: 0.4338100 (2250)	total: 10m 56s	remaining: 1m 12s
2499:	learn: 0.3576162	test: 0.4323179	best: 0.4323179 

In [8]:
# 7. Оценка качества

# Получаем предсказания в логарифмах
log_preds = model.predict(X_val)

# Конвертируем обратно в рубли (exp(x) - 1)
real_preds = np.expm1(log_preds)
real_preds = np.maximum(real_preds, 0) # Страховка от отрицательных значений

# Считаем финальную метрику WMAE
wmae_score = weighted_mean_absolute_error(y_val, real_preds, w_val)
print(f"==========================================")
print(f"FINAL Validation WMAE: {wmae_score:.2f}")
print(f"==========================================")

FINAL Validation WMAE: 38920.82


In [9]:
# 8. Сохранение для Сервиса и Сабмита

# A. Сохраняем модель
model.save_model("model.cbm")

# B. Сохраняем конфиг признаков (важно для VS Code сервиса)
metadata = {
    "feature_names": feature_names,
    "cat_features": cat_features
}
with open("features.json", "w") as f:
    json.dump(metadata, f)

print("Файлы model.cbm и features.json сохранены. Скачайте их для сервиса!")

# C. Генерируем сабмит для хакатона
log_submit_preds = model.predict(X_submit)
submit_preds = np.expm1(log_submit_preds) # Не забываем expm1
submit_preds = np.maximum(submit_preds, 0)

submission = test_df[['id']].copy()
submission['target'] = submit_preds
submission.to_csv('submission_optimized.csv', index=False)

print("Файл submission_optimized.csv готов к отправке на лидерборд.")

Файлы model.cbm и features.json сохранены. Скачайте их для сервиса!
Файл submission_optimized.csv готов к отправке на лидерборд.
